In [1]:
include("../ToolBox/ToolBox.jl")
using .ToolBox
import EzXML
import XMLDict

In [4]:
function slc_calibration(path,t_0)
    doc = EzXML.readxml(path)
    cal_dict = XMLDict.xml_dict(doc)

    s1_cal  = Dict{String,Any}()

    abs_cal_constant = cal_dict["calibration"]["calibrationInformation"]["absoluteCalibrationConstant"]
    s1_cal["abs_cal_constant"] = parse(Float64,abs_cal_constant)

    cal_vectors = cal_dict["calibration"]["calibrationVectorList"]["calibrationVector"]
    s1_cal["azimuth_times"] = [Load._str_date2float(elem["azimuthTime"], t_0) for elem in cal_vectors]
    s1_cal["line"] = [parse(Int,elem["line"]) for elem in cal_vectors] .+1
    s1_cal["sample"] = parse.(Int,split(cal_vectors[1]["pixel"][""])) .+1
    s1_cal["sigma"] = vcat([parse.(Float64,split(elem["sigmaNought"][""]))' for elem in cal_vectors]...)
    s1_cal["beta"] = vcat([parse.(Float64,split(elem["betaNought"][""]))' for elem in cal_vectors]...)
    s1_cal["gamma"] = vcat([parse.(Float64,split(elem["gamma"][""]))' for elem in cal_vectors]...)
    s1_cal["dn"] = vcat([parse.(Float64,split(elem["dn"][""]))' for elem in cal_vectors]...)
    return s1_cal
end

slc_calibration (generic function with 1 method)

In [5]:
cal_path = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190917T170931_20190917T170958_029064_034C5E_2484.SAFE/annotation/calibration/calibration-s1a-iw2-slc-vv-20190917t170932-20190917t170957-029064-034c5e-005.xml"
meta_path = "/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190917T170931_20190917T170958_029064_034C5E_2484.SAFE/annotation/s1a-iw2-slc-vv-20190917t170932-20190917t170957-029064-034c5e-005.xml"

"/Users/simon/Data/Sentinel/vejle_oktober/S1A_IW_SLC__1SDV_20190917T170931_20190917T170958_029064_034C5E_2484.SAFE/annotation/s1a-iw2-slc-vv-20190917t170932-20190917t170957-029064-034c5e-005.xml"

In [6]:
meta = Load.slc_meta(meta_path);
cal = slc_calibration(cal_path,meta["t_0"])

Dict{String,Any} with 8 entries:
  "sigma"            => [309.839 309.801 … 290.871 290.851; 309.846 309.807 … 2…
  "line"             => [-627, 27, 514, 1000, 1655, 2141, 2628, 3282, 3769, 425…
  "abs_cal_constant" => 1.0
  "dn"               => [237.0 237.0 … 237.0 237.0; 237.0 237.0 … 237.0 237.0; …
  "gamma"            => [279.022 278.97 … 251.537 251.506; 279.03 278.978 … 251…
  "sample"           => [1, 41, 81, 121, 161, 201, 241, 281, 321, 361  …  25601…
  "azimuth_times"    => [31.4712, 32.4712, 33.4712, 34.4712, 35.4712, 36.4712, …
  "beta"             => [237.0 237.0 … 237.0 237.0; 237.0 237.0 … 237.0 237.0; …

In [8]:
meta["mission_id"]

"S1A"